In [28]:
import tensorflow as tf
import numpy as np
#import matplotlib.plot as pyplt
import pandas as pd
import time
localtime = time.asctime( time.localtime(time.time()) )
print (localtime)

Wed Apr 13 23:26:45 2016


Read train and test data

In [10]:
train_file = 'train.csv'
test_file = 'test.csv'
train = pd.read_csv(train_file)
test = pd.read_csv(test_file)

In [53]:
image_size = 28
num_labels = 10
if isinstance(train, pd.DataFrame):
    train = train.as_matrix()
if isinstance(test, pd.DataFrame):
    test_dataset = test.as_matrix().astype(np.float32)
print (train.shape)
    
def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

train_num, img_size= train.shape
train_dataset = train[:,:-1]
train_labels = train[:, -1]


train_dataset, train_labels = reformat(train_dataset, train_labels)

print (train_dataset.shape)
print (train_labels.shape)
print (type(train_dataset))
print (test_dataset.shape)

(42000, 785)
(42000, 784)
(42000, 10)
<type 'numpy.ndarray'>
(28000, 784)


In [57]:
batch_size = 1000
n_hidden = 1024
num_labels = 10

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, train_dataset.shape[1]))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))

  tf_test_dataset = tf.constant(test_dataset)
  tf_hidden_dataset = tf.placeholder(tf.float32, shape = (batch_size, n_hidden))
    
  

  # Variables.
  weights_0 = tf.Variable(
    tf.truncated_normal([train_dataset.shape[1], n_hidden]))
  biases_0 = tf.Variable(tf.zeros([n_hidden]))
    
  weights_1 = tf.Variable(tf.truncated_normal([n_hidden, num_labels]))
  biases_1 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden_level = tf.matmul(tf_train_dataset, weights_0) + biases_0
  tf_hidden_dataset = tf.nn.relu(hidden_level)
  logits = tf.matmul(tf_hidden_dataset, weights_1) + biases_1
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  

  def multilayer_perceptron(x, w_0, b_0,w_1, b_1):
        l1 = tf.nn.relu(tf.matmul(x, w_0) + b_0)
        pred = tf.matmul(l1, w_1) + b_1
        return pred
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
 
  test_prediction = tf.nn.softmax(multilayer_perceptron(tf_test_dataset, weights_0, biases_0, weights_1, biases_1))
  print (r'Run the following model!')

Run the following model!


In [60]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [63]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size),:]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
    
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
        
  print("Train accuracy: %.1f%%" % accuracy(train_prediction.eval(), train_labels))
  test_prediction = test_prediction.eval()
  print (type(test_prediction))
  print (test_prediction.shape)
  out = pd.DataFrame(np.argmax(test_prediction,1))
  out.to_csv('out.csv')

Initialized
Minibatch loss at step 0: 43041.148438
Minibatch accuracy: 10.0%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%


InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder' with dtype float and shape [1000,784]
	 [[Node: Placeholder = Placeholder[dtype=DT_FLOAT, shape=[1000,784], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]
Caused by op u'Placeholder', defined at:
  File "/usr/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelapp.py", line 442, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python2.7/dist-packages/zmq/eventloop/ioloop.py", line 162, in start
    super(ZMQIOLoop, self).start()
  File "/usr/l